In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
os.environ["OPENCV_IO_ENABLE_OPENEXR"]="1"

import tensorflow as tf

2022-11-13 10:25:10.278922: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/data/awong189/miniconda/envs/thesis/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def freeze_graph(graph, session, output, save_pb_dir='.', save_pb_name='frozen_model.pb', save_pb_as_text=False):
    with graph.as_default():
        graphdef_inf = tf.graph_util.remove_training_nodes(graph.as_graph_def())
        graphdef_frozen = tf.graph_util.convert_variables_to_constants(session, graphdef_inf, output)
        graph_io.write_graph(graphdef_frozen, save_pb_dir, save_pb_name, as_text=save_pb_as_text)
        return graphdef_frozen

In [12]:
from utils import new_new_loss, loss_function
tf.keras.backend.clear_session()
tf.keras.backend.set_learning_phase(0) 

custom_func = {"new_new_loss": new_new_loss, "loss_function": loss_function}
model = tf.keras.models.load_model('./mobile_scene_skip_loss_1.h5', custom_objects=custom_func)
model.summary()

# inputs = model.input
# output = model.get_layer("block_13_expand_relu").output
# x = tf.keras.layers.Conv2D(filters=1, activation="sigmoid", kernel_size=3, padding="same")(output)

# model = tf.keras.Model(inputs=inputs, outputs=x)

/data/awong189/miniconda/envs/thesis/lib/python3.9/site-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 640, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 128, 320, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 128, 320, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 128, 320, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 640, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 128, 320, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 128, 320, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 128, 320, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [5]:
tf.saved_model.save(model, "mobile_scene_skip_loss_1")

2022-11-13 10:25:47.972967: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: mobile_scene_skip_loss_1/assets


In [13]:
!python3 -m tf2onnx.convert --saved-model mobile_scene_skip_loss_1 --output "mobile_scene_skip_loss_1.onnx"

2022-11-13 11:29:08.780203: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
/data/awong189/miniconda/envs/thesis/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/data/awong189/miniconda/envs/thesis/lib/python3.9/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2022-11-13 11:29:10.229577: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-13 11:29:10.230705: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-13 11:29:10.870710: I t

2022-11-13 11:29:17.146758: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 990 nodes (753), 1435 edges (1198), time = 26.497ms.
  function_optimizer: function_optimizer did nothing. time = 0.473ms.

2022-11-13 11:29:17.866311: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-13 11:29:17.867335: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:42:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-11-13 11:29:17.867388: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-13 11:29:17.867423: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic l

In [14]:
!python -m onnxruntime.quantization.preprocess --input "mobile_scene_skip_loss_1.onnx" --output "mobile_scene_skip_loss_infer_1.onnx"

In [15]:
import numpy
import onnxruntime
import os
from onnxruntime.quantization import CalibrationDataReader
import loader
import data

pano_path = "/data3/awong/pano/M3D_low/"
train = loader.generate_dataframe("./splits/M3D_v1_train.yaml", pano_path)
train_generator = data.DataGenerator(
            train, batch_size=32, shuffle=False, wrap=64
        )

class MobileNetDataReader(CalibrationDataReader):
    def __init__(self, model_path):
        self.enum_data = None

#         # Use inference session to get input shape.
        session = onnxruntime.InferenceSession(model_path, None)
        (_, _, height, width) = session.get_inputs()[0].shape
        self.input_name = session.get_inputs()[0].name
#         self.datasize = len(train.images)

    def get_next(self):
        if self.enum_data is None:
            self.enum_data = iter(train_generator)
        
        vals = next(self.enum_data, None)
        if vals:
            return {self.input_name: vals[0]}
        return None

    def rewind(self):
        self.enum_data = None

In [16]:
from onnxruntime.quantization import quantize_static, QuantType, QuantFormat

input_model_path = 'mobile_scene_skip_loss_infer_1.onnx'
output_model_path = 'mobile_scene_skip_loss_quant.onnx'
dr = MobileNetDataReader(input_model_path)

# print(dr.get_next())

quantize_static(
        input_model_path,
        output_model_path,
        dr,
        quant_format=QuantFormat.QDQ,
        per_channel=False,
        weight_type=QuantType.QInt8,
        optimize_model=False,
    )

In [25]:
onnxruntime.__version__

'1.11.0'